In [1]:
using POMDPPolicies
using POMDPs
using BeliefUpdaters
using POMDPModels

In [2]:
struct HeuristicPolicy{P<:POMDP, A} <: Policy
    pomdp::P
    action_map::Vector{A}
end

In [3]:
function HeuristicPolicy(pomdp::POMDP)
    HeuristicPolicy(pomdp, actions(pomdp))
end

HeuristicPolicy

In [4]:
DiscreteUpdater(pomdp)

UndefVarError: UndefVarError: pomdp not defined

In [5]:
function action(p::HeuristicPolicy, b::DiscreteBelief)
    max_value = -Inf
    best_idx = 1
    for i = 1:n_actions(p.pomdp)
        a = p.action_map[i]
        action_val = 0.0
        for (bel,state) in zip(b.b, b.state_list)
            action_val += bel*reward(p.pomdp, state, a)
        end
        
        if action_val > max_value
            best_idx = i
            max_value = action_val
        end
    end
    
    return p.action_map[best_idx]
end

action (generic function with 1 method)

In [6]:
pomdp = TigerPOMDP()

TigerPOMDP(-1.0, -100.0, 10.0, 0.85, 0.95)

In [7]:
heur_pol = HeuristicPolicy(pomdp)

HeuristicPolicy{TigerPOMDP,Int64}(TigerPOMDP(-1.0, -100.0, 10.0, 0.85, 0.95), [0, 1, 2])

In [8]:
using POMDPSimulators
rollout_sim = RolloutSimulator(max_steps=10);
history_fib = simulate(rollout_sim, pomdp, heur_pol, DiscreteUpdater(pomdp));

MethodError: MethodError: no method matching action(::HeuristicPolicy{TigerPOMDP,Int64}, ::DiscreteBelief{TigerPOMDP,Bool})
Closest candidates are:
  action(!Matched::CategoricalTabularPolicy, ::Any) at /home/shushman/.julia/packages/POMDPPolicies/oW6ud/src/stochastic.jl:30
  action(!Matched::EpsGreedyPolicy, ::Any) at /home/shushman/.julia/packages/POMDPPolicies/oW6ud/src/stochastic.jl:45
  action(!Matched::FeedWhenCrying, ::Any) at /home/shushman/.julia/packages/POMDPModels/dUyB8/src/CryingBabies.jl:81
  ...